# Hybrid Approach
- Deep learning to localize the sign using YOLO
- classical ML to recognize the traffic sign using SVM

## Imports

In [2]:
import os
import shutil
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage.feature import hog
import joblib

c:\Users\OSAMA\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset Manipulation

### dataset to YOLO format

In [ ]:
# Paths
root_dir = "YOLODataset"
ts_dir = os.path.join(root_dir, "ts")
train_txt = os.path.join(root_dir, "train.txt")
val_txt = os.path.join(root_dir, "test.txt")

# Output YOLO structure
output_dir = "YOLO_dataset"
images_train = os.path.join(output_dir, "images/train")
images_val = os.path.join(output_dir, "images/val")
labels_train = os.path.join(output_dir, "labels/train")
labels_val = os.path.join(output_dir, "labels/val")

# Create directories
os.makedirs(images_train, exist_ok=True)
os.makedirs(images_val, exist_ok=True)
os.makedirs(labels_train, exist_ok=True)
os.makedirs(labels_val, exist_ok=True)

def process_split(txt_path, image_dst, label_dst):
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        image_path = line.strip()
        image_name = os.path.basename(image_path)
        label_name = os.path.splitext(image_name)[0] + ".txt"

        full_image_path = os.path.join(ts_dir, image_name)
        full_label_path = os.path.join(ts_dir, label_name)

        # Copy image
        shutil.copy(full_image_path, os.path.join(image_dst, image_name))

        # Copy label
        if os.path.exists(full_label_path):
            shutil.copy(full_label_path, os.path.join(label_dst, label_name))
        else:
            print(f"⚠️ Label not found for {image_name}")

# Process train and val splits
process_split(train_txt, images_train, labels_train)
process_split(val_txt, images_val, labels_val)

# Create data.yaml
data_yaml_path = os.path.join(output_dir, "data.yaml")
with open(data_yaml_path, "w") as f:
    f.write(f"""train: {os.path.abspath(images_train)}
val: {os.path.abspath(images_val)}

nc: 4
names: ['prohibitory', 'danger', 'mandatory', 'other']
""")

print(" Dataset prepared at:", output_dir)


✅ Dataset prepared at: YOLO_dataset


### filtered dataset

In [ ]:
# Original dataset class names
ORIGINAL_NAMES = [
    'Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110',
    'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50',
    'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop'
]

# Mapping original class indices → new class indices
CLASS_MAP = {
    0: 4,  # Green Light → traffic light
    1: 4,  # Red Light → traffic light
    2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0  # Speed Limits + Stop → prohibitory
    # Other classes (danger, mandatory, etc.) can be added similarly if needed
}

# Output dataset structure
original_root = Path("car_yolo_format")   # original dataset
new_root = Path("car_yolo_filtered")      # new filtered dataset
splits = ["train", "val", "test"]

for split in splits:
    img_dir = original_root / split / "images"
    lbl_dir = original_root / split / "labels"

    new_img_dir = new_root / split / "images"
    new_lbl_dir = new_root / split / "labels"
    new_img_dir.mkdir(parents=True, exist_ok=True)
    new_lbl_dir.mkdir(parents=True, exist_ok=True)

    for lbl_file in lbl_dir.glob("*.txt"):
        new_lines = []
        with open(lbl_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                original_cls = int(parts[0])
                bbox = parts[1:]

                # Filter and remap class
                if original_cls in CLASS_MAP:
                    new_cls = CLASS_MAP[original_cls]
                    new_lines.append(f"{new_cls} {' '.join(bbox)}\n")

        if new_lines:
            # Save filtered label
            new_lbl_path = new_lbl_dir / lbl_file.name
            with open(new_lbl_path, "w") as f:
                f.writelines(new_lines)

            # Copy image
            img_path = img_dir / (lbl_file.stem + ".jpg")
            if img_path.exists():
                shutil.copy(img_path, new_img_dir / img_path.name)

# Save new data.yaml
with open(new_root / "data.yaml", "w") as f:
    f.write("""train: ../train/images
val: ../val/images
test: ../test/images

nc: 5
names: ['prohibitory', 'danger', 'mandatory', 'other', 'traffic light']
""")

print(" New YOLO dataset created with only selected classes at:", new_root)


✅ New YOLO dataset created with only selected classes at: car_yolo_filtered


### Traffic light dataset

In [ ]:
# Traffic light class indices in original dataset
TRAFFIC_LIGHT_CLASSES = [0, 1]  # Green Light and Red Light in original data

# Dataset paths
original_root = Path("car_yolo_format")  # original dataset
new_root = Path("traffic_light_dataset")  # new dataset only with traffic lights
splits = ["train", "val", "test"]

for split in splits:
    img_dir = original_root / split / "images"
    lbl_dir = original_root / split / "labels"

    new_img_dir = new_root / split / "images"
    new_lbl_dir = new_root / split / "labels"
    new_img_dir.mkdir(parents=True, exist_ok=True)
    new_lbl_dir.mkdir(parents=True, exist_ok=True)

    for lbl_file in lbl_dir.glob("*.txt"):
        new_lines = []
        with open(lbl_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls = int(parts[0])
                bbox = parts[1:]

                if cls in TRAFFIC_LIGHT_CLASSES:
                    new_lines.append(f"0 {' '.join(bbox)}\n")  # remap to class 0

        if new_lines:
            # Save label
            new_lbl_path = new_lbl_dir / lbl_file.name
            with open(new_lbl_path, "w") as f:
                f.writelines(new_lines)

            # Copy image
            img_path = img_dir / (lbl_file.stem + ".jpg")
            if img_path.exists():
                shutil.copy(img_path, new_img_dir / img_path.name)

# Save new data.yaml
with open(new_root / "data.yaml", "w") as f:
    f.write("""train: ../train/images
val: ../val/images
test: ../test/images

nc: 1
names: ['traffic light']
""")

print(" New traffic light-only dataset created at:", new_root)


✅ New traffic light-only dataset created at: traffic_light_dataset


### change the traffic light class

In [ ]:
import os
from pathlib import Path

# 🔁 Path to your YOLO dataset labels
label_dir = Path("C:/Users/OSAMA/Downloads/erp_traffic_light.v1i.yolov8/train/labels")

for txt_file in label_dir.glob("*.txt"):
    updated_lines = []

    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls = int(parts[0])
            # Change class 0 to class 4
            if cls == 0:
                parts[0] = "4"
            updated_lines.append(" ".join(parts) + "\n")

    # Overwrite with updated content
    with open(txt_file, "w") as f:
        f.writelines(updated_lines)

print("All class 0 labels changed to class 4.")


All class 0 labels changed to class 4.


# Train YOLO detector

In [ ]:
model = YOLO("yolov8n.yaml").load("yolov8n.pt")  # build from YAML and transfer weights


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [13]:
model.train(data="YOLO_dataset/data.yaml", epochs=5, imgsz=960, batch=16)

New https://pypi.org/project/ultralytics/8.3.133 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=YOLO_dataset/data.yaml, epochs=5, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, vis

In [ ]:
model = YOLO("runs/detect/train8/weights/best.pt")
model.train(data="YOLO_dataset/data.yaml", epochs=2, imgsz=960,visualize=True , batch=16, degrees=5, translate=0.1, scale=0.5, shear=2.0, flipud=0.5, fliplr=0.5, mosaic=1.0, mixup=0.1)

New https://pypi.org/project/ultralytics/8.3.133 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=runs/detect/train8/weights/best.pt, data=YOLO_dataset/data.yaml, epochs=2, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
model = YOLO("runs/detect/train9/weights/best.pt")
model.train(data="car_yolo_filtered/data.yaml", epochs=2, imgsz=960,visualize=True , batch=16)

New https://pypi.org/project/ultralytics/8.3.135 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=runs/detect/train9/weights/best.pt, data=car_yolo_filtered/data.yaml, epochs=2, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_str

## predict

In [ ]:
# Class names for your model (match your `data.yaml`)
CLASS_NAMES = ['prohibitory', 'danger', 'mandatory', 'other']

# Load trained model
model = YOLO("runs/detect/train13/weights/best.pt")

# Load image
img_path = "Dataset/Train/1/00001_00000_00001.png"

# resize image
image = cv2.imread(img_path)
image = cv2.resize(image, (1300, 960))
# Save resized image
resized_img_path = "C:/Users/OSAMA/Desktop/download_resized.jpg"
cv2.imwrite(resized_img_path, image)
# Load resized image
img_path = resized_img_path

# Run prediction
results = model.predict(source=img_path, conf=0.5)

# Get first result
boxes = results[0].boxes

# Draw bounding boxes
for box in boxes:
    # Get box coordinates (xyxy format: [x1, y1, x2, y2])
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cls_id = int(box.cls[0])
    conf = float(box.conf[0])
    label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"

    print(f"Class: {label}, BBox: {x1, y1, x2, y2}")

    # Draw box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Show image
cv2.imshow("Detected Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\OSAMA\Desktop\download_resized.jpg: 736x960 1 prohibitory, 217.4ms
Speed: 7.2ms preprocess, 217.4ms inference, 5.6ms postprocess per image at shape (1, 3, 736, 960)


Class: prohibitory 0.78, BBox: (0, 120, 1147, 849)


In [6]:
# Class names for your model (match your `data.yaml`)
CLASS_NAMES = ['prohibitory', 'danger', 'mandatory', 'other', 'Traffic light']

# Load trained model
model = YOLO("runs/detect/train14/weights/best.pt")

# Load video (replace with your file path or 0 for webcam)
video_path = "C:/osama/CUFE/5-Senior2/GP/ADAS-System-prototype/Traffic_Recognition/CarlaVideos/traffic signs test.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Optional: Save the output video
out = cv2.VideoWriter("C:/Users/OSAMA/Desktop/output_detected.mp4",
                      cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (1200, 960))

    # Run YOLO prediction
    results = model.predict(source=frame, conf=0.4, verbose=False)

    # Draw bounding boxes
    boxes = results[0].boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"

        print(f"Class: {label}, BBox: {x1, y1, x2, y2}")

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow("Traffic Sign Detection", frame)

    # Save the frame
    out.write(frame)

    # Break loop on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()


Class: prohibitory 0.93, BBox: (749, 393, 803, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.93, BBox: (749, 393, 803, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.94, BBox: (749, 393, 803, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.94, BBox: (749, 393, 804, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.94, BBox: (749, 393, 803, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.94, BBox: (749, 393, 803, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.93, BBox: (749, 393, 804, 473)
Class: prohibitory 0.53, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.93, BBox: (749, 393, 803, 473)
Class: prohibitory 0.50, BBox: (1099, 550, 1143, 617)
Class: prohibitory 0.42, BBox: (1099, 521, 1145, 617)
Class: prohibitory 0.93, BBox: (749, 393, 803, 473)
Class: prohibitory 0.51, BBox: (1099, 550, 114

KeyboardInterrupt: 

# SVM

### Preprocessing the dataset for SVM

In [ ]:
# CONFIGURATION
dataset_root = Path("car_yolo_format")
save_root = Path("svm_dataset")
save_root.mkdir(parents=True, exist_ok=True)

# Class names
class_names = [
    'Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110',
    'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50',
    'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop'
]

# Loop over train/val/test
for split in ["train", "val", "test"]:
    image_dir = dataset_root / split / "images"
    label_dir = dataset_root / split / "labels"

    for img_file in image_dir.glob("*.jpg"):
        image = cv2.imread(str(img_file))
        if image is None:
            continue
        h, w = image.shape[:2]

        label_file = label_dir / (img_file.stem + ".txt")
        if not label_file.exists():
            continue

        with open(label_file, "r") as f:
            for i, line in enumerate(f.readlines()):
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                class_id, x_center, y_center, bw, bh = map(float, parts)
                class_id = int(class_id)

                # Convert from YOLO normalized format to pixel coordinates
                x1 = int((x_center - bw / 2) * w)
                y1 = int((y_center - bh / 2) * h)
                x2 = int((x_center + bw / 2) * w)
                y2 = int((y_center + bh / 2) * h)

                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(w, x2)
                y2 = min(h, y2)

                crop = image[y1:y2, x1:x2]
                if crop.size == 0:
                    continue

                crop = cv2.resize(crop, (64, 64))
                out_dir = save_root / split / str(class_id)
                out_dir.mkdir(parents=True, exist_ok=True)

                out_path = out_dir / f"{img_file.stem}_{i}.jpg"
                cv2.imwrite(str(out_path), crop)

print(" preprocessed images saved for SVM training.")


✅ Cropped images saved for SVM training.


### Features Extraction

In [ ]:
from pathlib import Path
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

def load_data_and_extract_features(root_dir):
    X, y = [], []
    for class_dir in Path(root_dir).glob("*"):
        if not class_dir.is_dir():
            continue
        label = int(class_dir.name)
        for img_path in class_dir.glob("*.jpg"):
            img = cv2.imread(str(img_path))
            if img is None:
                continue

            # Resize for consistency
            img = cv2.resize(img, (64, 64))

            # --- HOG ---
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            hog_feat = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

            # --- HSV ---
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)

            # Mean HSV values
            mean_h = np.mean(h)
            mean_s = np.mean(s)
            mean_v = np.mean(v)

            # Histogram of hue (8 bins)
            hue_hist = cv2.calcHist([h], [0], None, [8], [0, 180])
            hue_hist = cv2.normalize(hue_hist, hue_hist).flatten()

            # Combine features
            full_feature = np.concatenate([hog_feat, [mean_h, mean_s, mean_v], hue_hist])
            X.append(full_feature)
            y.append(label)

    return np.array(X), np.array(y)


In [ ]:
# Load training data
X, y = load_data_and_extract_features("svm_dataset/train")

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))

### Train

In [ ]:
# Load crops from svm_dataset/train (cropped by class ID)
def load_data(root_dir):
    X, y = [], []
    for class_dir in Path(root_dir).glob("*"):
        if not class_dir.is_dir():
            continue
        label = int(class_dir.name)
        for img_path in class_dir.glob("*.jpg"):
            img = cv2.imread(str(img_path))
            if img is None:
                continue
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Resize to 64x64
            gray = cv2.resize(gray, (64, 64))
            # Extract HOG features
            features = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

# Load training data
X, y = load_data("svm_dataset/train")

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Validate
y_pred = clf.predict(X_val)
report = classification_report(y_val, y_pred, output_dict=False)

report


'              precision    recall  f1-score   support\n\n           0       0.79      0.88      0.83       216\n           1       0.70      0.69      0.70       128\n           2       0.75      1.00      0.86         3\n           3       1.00      0.92      0.96        53\n           4       0.94      0.79      0.86        19\n           5       0.85      0.95      0.90        42\n           6       0.91      0.90      0.91        59\n           7       0.94      0.92      0.93       111\n           8       0.97      0.90      0.93        63\n           9       0.91      0.89      0.90        44\n          10       0.97      0.90      0.93       101\n          11       0.89      0.89      0.89        71\n          12       0.89      0.83      0.86        69\n          13       0.85      0.87      0.86        79\n          14       1.00      1.00      1.00        77\n\n    accuracy                           0.87      1135\n   macro avg       0.89      0.89      0.89      1135\nweigh

### save the model

In [ ]:
joblib.dump(clf, "svm_model.pkl")

['svm_model.pkl']

### Final Prediction

In [5]:
# Load YOLO model
yolo_model = YOLO("runs/detect/train12/weights/best.pt")

# Load SVM model
svm_model = joblib.load("svm_model.pkl")

# Class names (must match your 15-class SVM dataset)
CLASS_NAMES = [
    'Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110',
    'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50',
    'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop'
]

# Open video
cap = cv2.VideoCapture("C:/osama/CUFE/5-Senior2/GP/ADAS-System-prototype/Traffic_Recognition/CarlaVideos/traffic signs test.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO
    results = yolo_model.predict(frame, conf=0.2)
    boxes = results[0].boxes

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped = frame[y1:y2, x1:x2]

        # Check for valid crop
        if cropped.size == 0 or x2 - x1 < 10 or y2 - y1 < 10:
            continue

        # Resize and convert to gray
        resized = cv2.resize(cropped, (64, 64))
        gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

        # Extract HOG features
        features = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True).reshape(1, -1)

        # Predict using SVM
        prediction = svm_model.predict(features)[0]
        confidence = max(svm_model.predict_proba(features)[0])

        # Filter out weak detections
        if confidence < 0.9:
            continue

        label = f"{CLASS_NAMES[prediction]} ({confidence:.2f})"

        # Draw result
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("YOLO + SVM Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 544x960 1 prohibitory, 152.6ms
Speed: 2.4ms preprocess, 152.6ms inference, 5.7ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 146.4ms
Speed: 9.1ms preprocess, 146.4ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 141.7ms
Speed: 4.0ms preprocess, 141.7ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 138.3ms
Speed: 5.1ms preprocess, 138.3ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 142.2ms
Speed: 7.0ms preprocess, 142.2ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 145.3ms
Speed: 6.7ms preprocess, 145.3ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 144.8ms
Speed: 4.8ms preprocess, 144.8ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 prohibitory, 142.3ms
Speed: 3.6ms preprocess, 142.3ms i

KeyboardInterrupt: 